# Data Exploration and Preprocessing

# 1. Importing the Libraries and  Dataset

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('/content/MLE-Assignment.csv')

# 2. Basic Info and Overview of the Dataset


In [ ]:
print("\n Viewing First 5 Rows : ")
data.head(5)

In [ ]:
print("\n Basic Dataset Info : ")
print(data.info())

# 3.Checking Data Distribution  

In [ ]:
numeric_cols = data.select_dtypes(include=['number']).columns

# Dynamically calculate number of rows & columns
num_plots = len(numeric_cols)
num_cols = 3
num_rows = int(np.ceil(num_plots / num_cols))

plt.figure(figsize=(12, 4 * num_rows))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(num_rows, num_cols, i)
    sns.histplot(data[col], kde=True, bins=30)
    plt.title(col)

plt.tight_layout()
plt.show()

# 4.Checking Missing Values in the Dataset

In [ ]:
print("\n Checking Missing Values :")
print(data.isnull().sum())

In [ ]:
data.replace(["", " ", "NaN", "NULL"], np.nan, inplace=True)
print(data.isnull().sum())


# 5. Checking and Removing Duplicates

In [ ]:
duplicates = data.duplicated().sum()
print(f"Number of Duplicates Present in the Dataset is : ", {duplicates})

# 6. Outlier Detection

In [ ]:
numeric_cols = data.select_dtypes(include=['number']).columns

# Dynamically determine rows and columns
num_plots = len(numeric_cols)
num_cols = 3  # Keep columns fixed at 3
num_rows = int(np.ceil(num_plots / num_cols))

plt.figure(figsize=(12, 4 * num_rows))
for i, col in enumerate(numeric_cols, 1):
    plt.subplot(num_rows, num_cols, i)
    sns.boxplot(y=data[col])
    plt.title(col)

plt.tight_layout()
plt.show()

In [ ]:
numeric_data = data.select_dtypes(include=['number'])

# Compute IQR
Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1

outliers = ((numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR)))
outlier_counts = outliers.sum()
print("Outlier count per column:\n", outlier_counts)

In [ ]:
from scipy.stats.mstats import winsorize

for col in numeric_data.columns:
    data[col] = winsorize(data[col], limits=[0.05, 0.05])  # Capping at 5% on both sides


# 7. Outlier Detection in DON Concentration

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x=data['vomitoxin_ppb'], color="red")
plt.title("Outlier Detection in DON Concentration")
plt.xlabel("DON Concentration (ppb)")
plt.show()

# 7.1 Analyzing Outliers Impact
If the target variable is heavily skewed, then applying log transformation instead of removing outliers

In [ ]:
#Checking The Distribution of Target Variable
plt.figure(figsize=(8, 5))
sns.histplot(data['vomitoxin_ppb'], bins=30, kde=True)
plt.xlabel('DON Concentraion(ppd)')
plt.ylabel('Frequency')
plt.title('Distribution of DON Concentration')
plt.show()

In [ ]:
#calculate skewness
from scipy.stats import skew
skewness = skew(data['vomitoxin_ppb'])
print(f"Skewness of DON Concentration : {skewness:.2f}")

In [ ]:
# Checking if skewness is high
if abs(skewness) > 1:
    print("The data is highly skewed. A log transformation might help.")
elif abs(skewness) > 0.5:
    print("The data is moderately skewed.")
else:
    print("The data is approximately normal.")

# 7.2 Applying Log Transformation
For reducing skewness while preserving important variations

In [ ]:
data['log_vomitoxin_ppd'] = np.log1p(data['vomitoxin_ppb'])

In [ ]:
#plotting histogram after log transformation
plt.figure(figsize=(8,5))
sns.histplot(data['log_vomitoxin_ppd'], bins=30, kde=True)
plt.xlabel('Log Transformed DON Concentration(ppd)')
plt.ylabel('Frequency')
plt.title('Distribution of Log Transformed DON Concentration')
plt.show()

In [ ]:
new_skewness = skew(data['log_vomitoxin_ppd'])
print(f"Skewness After Log Transformation : {new_skewness:.2f}")

In [ ]:
# Checking if skewness is high
if abs(new_skewness) > 1:
    print("The data is highly skewed. A log transformation might help.")
elif abs(new_skewness) > 0.5:
    print("The data is moderately skewed.")
else:
    print("The data is approximately normal.")

# 8. Standardizing or Normalizing the Spectral Data.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

numeric_data = data.select_dtypes(include=['number'])

# Apply scaling only to numeric columns
scaler = StandardScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data[numeric_cols]), columns=numeric_cols)

# Keep non-numeric columns and merge back
data_scaled = pd.concat([data.drop(columns=numeric_cols), scaled_data], axis=1)

print("Scaling applied successfully!")

In [ ]:
non_spectral_columns = ["hsi_id", "vomitoxin_ppb"]
spectral_columns = [col for col in data.columns if col not in non_spectral_columns]


scaler = StandardScaler()
data[spectral_columns] = scaler.fit_transform(data[spectral_columns])


# 9. Generating Visualizations

9.1 Line Plot for Average Reflectance Over Wavelengths

In [ ]:

avg_reflectance = data[spectral_columns].mean()
plt.figure(figsize=(10, 5))
plt.plot(avg_reflectance.index, avg_reflectance.values)
plt.xlabel("Wavelength")
plt.ylabel("Average Reflectance")
plt.title("Average Reflectance Over Wavelengths")
plt.show()

9.2 Heatmap for Correlation Analysis

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(data[spectral_columns].corr(), cmap="coolwarm", annot=False)
plt.title("Heatmap of Spectral Data")
plt.show()

9.3 Pairplot for Sample Comparisons

In [ ]:
sns.pairplot(data[spectral_columns[:5]])
plt.show()

9.4 Line Plot for Reflectance Over Wavelengths

In [ ]:
import matplotlib.pyplot as plt

# Assuming columns '0' to '447' are spectral bands
spectral_columns = [col for col in data.columns if col.isdigit()]

# Compute average reflectance across samples
avg_reflectance = data[spectral_columns].mean()

# Plot reflectance trend
plt.figure(figsize=(12, 5))
plt.plot(avg_reflectance.index, avg_reflectance.values, marker='o', linestyle='-', label="Avg Reflectance")
plt.xlabel("Wavelength Index")
plt.ylabel("Average Reflectance")
plt.title("Average Reflectance Over Wavelengths")
plt.legend()
plt.show()


#10. Advanced  Data Quality Checks

10.1 Automated Sensor Drift & Data Consistency Checks

In [ ]:
from scipy.stats import zscore

# Compute Reflectance Differences (First-Order Derivative Check)
reflectance_diff = data[spectral_columns].diff(axis=1).mean(axis=1)

# Handle NaN values if they exist
reflectance_diff.fillna(0, inplace=True)

# Efficiently join the column to avoid fragmentation issues
data = pd.concat([data, reflectance_diff.rename("reflectance_diff")], axis=1).copy()

# Confirm NaN values are handled
print("NaN count in reflectance_diff:", data["reflectance_diff"].isna().sum())
print(data["reflectance_diff"].describe())

# 🔹 Visualize Reflectance Change Over Wavelengths
plt.figure(figsize=(10, 5))
plt.plot(data.index, data["reflectance_diff"], label="Reflectance Change Rate", linestyle="dashed")
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel("Sample Index")
plt.ylabel("Reflectance Change")
plt.title("Sensor Drift Analysis")
plt.legend()
plt.show()


10.2 Feature Engineering: Spectral Indices (NDVI, NDWI, MSI)

In [ ]:
# Ensure required bands exist in dataset
if all(col in data.columns for col in ["70", "100", "140"]):
    # Compute spectral indices
    data["NDVI"] = (data["100"] - data["70"]) / (data["100"] + data["70"])
    data["NDWI"] = (data["100"] - data["140"]) / (data["100"] + data["140"])
    data["MSI"] = data["140"] / data["100"]

    print(" Spectral Indices (NDVI, NDWI, MSI) added successfully!")

    # Heatmap for Feature Correlation Analysis
    import seaborn as sns
    plt.figure(figsize=(8, 6))
    sns.heatmap(data[["NDVI", "NDWI", "MSI", "vomitoxin_ppb"]].corr(), annot=True, cmap="coolwarm")
    plt.title("Correlation of Spectral Indices with Target Variable")
    plt.show()
else:
    print(" Required spectral bands not found. Skipping spectral indices computation.")


# Saving the Preprocessed Data

In [ ]:
data.to_csv("preprocessed_data.csv", index=False)

print("Preprocessed data saved successfully!")

# Adding Logging to Data Preprocessing

In [ ]:
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Logging data shape and missing values
logging.info(f"Dataset shape after preprocessing: {data.shape}")
logging.info(f"Missing values after preprocessing:\n{data.isnull().sum()}")

# Log feature scaling or encoding
logging.info("Feature scaling and encoding completed.")


# Adding Unit Tests

In [ ]:
import unittest

def preprocess_data(df):
    if data.isnull().sum().sum() > 0:
        logging.warning("Dataset contains missing values!")
    return df.fillna(0)

# Define unit tests
class TestPreprocessing(unittest.TestCase):
    def test_missing_values(self):
        df_test = pd.DataFrame({"A": [1, 2, np.nan], "B": [3, np.nan, 5]})
        df_result = preprocess_data(df_test)
        self.assertFalse(df_result.isnull().values.any())

# Run tests
unittest.main(argv=[''], exit=False)

# Model Training

# 1. Installing required Libraries

In [ ]:
!pip install scikit-learn tensorflow optuna


In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import optuna

# 2. Load Preprocessed Data

In [ ]:
df = pd.read_csv('/content/preprocessed_data.csv')

# 3. Defining Features X and Target Variable Y

In [ ]:
x = df.drop(columns=['hsi_id','vomitoxin_ppb'])
y = df["vomitoxin_ppb"]

# 4. Feature Engineering (Mutual Information)

In [ ]:
from sklearn.feature_selection import mutual_info_regression, f_regression

mi_scores = mutual_info_regression(x, y)
mi_sorted = sorted(zip(x.columns, mi_scores), key=lambda x: x[1], reverse=True)
selected_mi_features = [feature for feature, score in mi_sorted[:10]]

In [ ]:
#  Feature Selection (ANOVA F-test)
f_scores, p_values = f_regression(x, y)
selected_f_features = x.columns[p_values < 0.05]

In [ ]:
# Select Common Features from Both Methods
selected_features = list(set(selected_mi_features) & set(selected_f_features))

In [ ]:
# Filter the Dataset with Selected Features
X_selected = x[selected_features]

# 5. Splitting Data into Training and Testing Variable

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state=42)

6. Normalizing the Features

In [ ]:
#  Normalize Features
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# 7. XG Boost Model

In [ ]:
from xgboost import XGBRegressor

In [ ]:
#  Train XGBoost Model
model = XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)
model.fit(x_train, y_train)

In [ ]:
#  Make Predictions
y_pred = model.predict(x_test)
print(y_pred)

# Model Evaluation

#1. Calculate regression metrics

In [ ]:
#  Evaluate Model Performance
from sklearn.metrics import r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print(f" MAE: {mae:.4f}, MSE: {mse:.4f}, R² Score: {r2:.4f}")


# 2. Visual Evaluation

2.1 Plotting a scatter plot comparing actual vs. predicted values

In [ ]:
#  Scatter Plot: Actual vs. Predicted
plt.figure(figsize=(8, 5))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Actual Vomitoxin_ppb")
plt.ylabel("Predicted Vomitoxin_ppb")
plt.title("Actual vs. Predicted Values (XGBoost)")
plt.show()

2.2 Performing Residual Analysis to identify any systematic errors.

In [ ]:
# Extract actual values (vomitoxin_ppd column)
actual_values = df["vomitoxin_ppb"].values

# Ensure actual & predicted values have the same length
actual_values = actual_values[:len(y_pred)]

# Create a DataFrame to store actual vs. predicted values
results_df = pd.DataFrame({"Actual": actual_values, "Predicted": y_pred})

# Save to CSV
csv_path = "actual_vs_predicted.csv"
results_df.to_csv(csv_path, index=False)

print(f"CSV file created: {csv_path}")

In [ ]:
actual_predicted_data = pd.read_csv('/content/actual_vs_predicted.csv')
print(actual_predicted_data.head())

In [ ]:
# Compute Residuals
residuals = actual_values - y_pred

In [ ]:
#  Residual Distribution
plt.figure(figsize=(8, 6))
sns.histplot(residuals, kde=True, bins=30, color="purple")
plt.axvline(0, color='red', linestyle='dashed', linewidth=1)
plt.xlabel("Residuals (Actual - Predicted)")
plt.ylabel("Frequency")
plt.title("Residual Distribution")
plt.show()

In [ ]:
# Residuals vs. Actual Values
plt.figure(figsize=(8, 6))
sns.scatterplot(x=actual_values, y=residuals, color='green', alpha=0.6)
plt.axhline(0, color='red', linestyle='dashed', linewidth=1)
plt.xlabel("Actual Vomitoxin_ppb")
plt.ylabel("Residuals")
plt.title("Residuals vs. Actual Values")
plt.show()

# Model Interpretablity and Expalinability

#1. Feature Importance

In [ ]:
import matplotlib.pyplot as plt
import xgboost as xgb

# Plot feature importance
xgb.plot_importance(model)
plt.title("Feature Importance (XGBoost)")
plt.show()


#2. Expalin Model Predictions With SHAP

In [ ]:
!pip install shap
import shap

# Initialize SHAP explainer
explainer = shap.Explainer(model, x_train)

# Compute SHAP values
shap_values = explainer(x_test)

# Summary plot - Shows feature importance globally
shap.summary_plot(shap_values, x_test)


# 3. Local Interpretability with LIME

 Install & Import LIME

In [ ]:
!pip install lime
from lime.lime_tabular import LimeTabularExplainer

Create LIME Explainer

In [ ]:
if isinstance(x_train, np.ndarray):
    x_train = pd.DataFrame(x_train, columns=[f"Feature_{i}" for i in range(x_train.shape[1])])

explainer = LimeTabularExplainer(
    training_data=np.array(x_train),
    feature_names=x_train.columns,
    mode="regression"
)


Choose a random sample

In [ ]:
if isinstance(x_test, np.ndarray):
    x_test = pd.DataFrame(x_test, columns=[f"Feature_{i}" for i in range(x_test.shape[1])])

i = 10  # Index of the sample to explain
exp = explainer.explain_instance(x_test.iloc[i], model.predict)

# Show explanation
exp.show_in_notebook()

# Saving the Trained Model

In [ ]:
import joblib

# Save the trained model
joblib.dump(model, "spectral_model.pkl")
print("Model saved successfully.")
